In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import os

In [2]:
driver_options = Options()
driver_options.add_argument("accept-language=en-US")

s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s,options=driver_options)
driver.maximize_window()


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\emman\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [3]:
INITIAL_WEBSITE = "https://www.jumia.sn/"
data = {}
# go to the root page and get header/menu with its sub-items

In [4]:
# Autres catégories -- None # issue to solve later

In [5]:
def process_article(article):
    """retrive needed information from an article"""
    
    name = article.find_element(by=By.CSS_SELECTOR, value='h3.name').text
    price = article.find_element(by=By.CSS_SELECTOR, value='div.prc').text
    img_link = article.find_element(by=By.CSS_SELECTOR, value='img.img').get_attribute('data-src')
    descrption_link = article.find_element(by=By.CSS_SELECTOR, value='a.core').get_attribute('href')
    try:
        old_price, decrease_percentage = article.find_element(by=By.CSS_SELECTOR, value='div.s-prc-w').text.split("\n")
    except:
        old_price, decrease_percentage = '', ''
    try:
        n_stars, n_votes = article.find_element(by=By.CSS_SELECTOR, value='div.rev').text.split("\n")
        # n_votes = int(n_votes.replace("(", "").replace(")", "")) # clean value and convert to int 
    except:
        n_stars, n_votes = '', ''
    try:
        article.find_element(by=By.CSS_SELECTOR, value='svg.ic.xprss')
        jumia_express = True
    except:
        jumia_express = False

    article_data = dict(name=name , price=price , descrption_link=descrption_link , img_link=img_link , jumia_express=jumia_express ,
                   old_price=old_price , decrease_percentage=decrease_percentage ,n_stars=n_stars, n_votes=n_votes  )
    return article_data

In [6]:
def close_popup(driver):
    try:
        [btn.click() for btn in driver.find_elements(by=By.CSS_SELECTOR, value='div#pop button.cls')]
    except:
        pass

In [7]:
# menu_items = driver.find_elements(by=By.CSS_SELECTOR, value="div.flyout-w a.itm")
# menu_items = { cat.find_element(by=By.CSS_SELECTOR, value='span').text:cat.get_attribute('href') for cat in menu_items}
# menu_items

In [8]:
driver.get(INITIAL_WEBSITE)
close_popup(driver)
menu_items = {}
menu_items_ = driver.find_elements(by=By.CSS_SELECTOR, value="div.flyout-w a.itm")
len(menu_items_)

12

In [9]:
for cat in menu_items_: 
    link = cat.get_attribute('href')

In [11]:
# a = menu_items_[0]
# a.aria_role

In [11]:
# cat.get_attribute('href')

In [ ]:
driver.get(INITIAL_WEBSITE)
close_popup(driver)
menu_items = {}
menu_items_ = driver.find_elements(by=By.CSS_SELECTOR, value="div.flyout-w a.itm")
for cat in menu_items_: 
    link = cat.get_attribute('href')
    category_name = cat.find_element(by=By.CSS_SELECTOR, value='span').text
    menu_items[category_name] = link
    
print(f"Available categories : {len(menu_items)} \n" ) 

for (category_name, category_link) in menu_items.items():
    print(f"{category_name} -- {category_link} " )
    if category_link is not None :
        last_page_scraped = False
        next_page = category_link

        while not last_page_scraped:
            # go to a page to scrape articles
            driver.get(next_page)
            close_popup(driver)
            # articles_ = driver.find_elements(by=By.CSS_SELECTOR, value='article.prd._fb.col.c-prd') # get article boxes
            # articles = [process_article(article) for article in articles_]
            articles = [ ]
            for item in driver.find_elements(by=By.CSS_SELECTOR, value='article.prd._fb.col.c-prd'): 
                # articles.append(process_article(article))
                name = item.find_element(by=By.CSS_SELECTOR, value='h3.name').text
                price = item.find_element(by=By.CSS_SELECTOR, value='div.prc').text
                img_link = item.find_element(by=By.CSS_SELECTOR, value='img.img').get_attribute('data-src')
                descrption_link = item.find_element(by=By.CSS_SELECTOR, value='a.core').get_attribute('href')
                try:
                    old_price, decrease_percentage = item.find_element(by=By.CSS_SELECTOR, value='div.s-prc-w').text.split("\n")
                except:
                    old_price, decrease_percentage = '', ''
                try:
                    n_stars, n_votes = item.find_element(by=By.CSS_SELECTOR, value='div.rev').text.split("\n")
                    # n_votes = int(n_votes.replace("(", "").replace(")", "")) # clean value and convert to int 
                except:
                    n_stars, n_votes = '', ''
                try:
                    item.find_element(by=By.CSS_SELECTOR, value='svg.ic.xprss')
                    jumia_express = True
                except:
                    jumia_express = False

                article_data = dict(name=name , price=price , descrption_link=descrption_link , img_link=img_link , jumia_express=jumia_express ,
                               old_price=old_price , decrease_percentage=decrease_percentage ,n_stars=n_stars, n_votes=n_votes  )
                print(article_data)
            # articles = [process_article(article) ]
            data[category_name] = articles 
            next_page_btn = driver.find_elements(by=By.CSS_SELECTOR, value='div.pg-w.-ptm.-pbxl a')[-2] 

            if next_page_btn.accessible_name not in ['Page suivante']:
                last_page_scraped = True

            # break

        n_results_str = driver.find_element(by=By.CSS_SELECTOR, value='p.-gy5.-phs').text
        print(f"Available articles in this category : {n_results_str.split()[0]} \n")
    
    # break

Available categories : 12 

Téléphones & Tablettes -- https://www.jumia.sn/telephone-tablette/ 
{'name': 'Samsung Galaxy A03 Core - 4G - 6.5" - RAM 2Go - ROM 32Go - Caméra 13+2MP - 5000mAh- Noir', 'price': '48 900 FCFA', 'descrption_link': 'https://www.jumia.sn/samsung-galaxy-a03-core-4g-6.5-ram-2go-rom-32go-camera-132mp-5000mah-noir-6100199.html', 'img_link': 'https://sn.jumia.is/unsafe/fit-in/300x300/filters:fill(white)/product/99/10016/1.jpg?8517', 'jumia_express': True, 'old_price': '75 000 FCFA', 'decrease_percentage': '35%', 'n_stars': '4 out of 5', 'n_votes': '(56)'}
{'name': 'Generic Montres Bluetooth Intelligentes Pour Les Bandes Ios Et Android', 'price': '5 390 FCFA', 'descrption_link': 'https://www.jumia.sn/generic-montres-bluetooth-intelligentes-pour-les-bandes-ios-et-android-9832502.html', 'img_link': 'https://sn.jumia.is/unsafe/fit-in/300x300/filters:fill(white)/product/20/52389/1.jpg?0039', 'jumia_express': True, 'old_price': '9 945 FCFA', 'decrease_percentage': '46%', '

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=102.0.5005.63)
Stacktrace:
Backtrace:
	Ordinal0 [0x004DD953+2414931]
	Ordinal0 [0x0046F5E1+1963489]
	Ordinal0 [0x0035C6B8+837304]
	Ordinal0 [0x0035F0B4+848052]
	Ordinal0 [0x0035EF72+847730]
	Ordinal0 [0x0035F200+848384]
	Ordinal0 [0x00389215+1020437]
	Ordinal0 [0x0038979B+1021851]
	Ordinal0 [0x0037FCF1+982257]
	Ordinal0 [0x003A44E4+1131748]
	Ordinal0 [0x0037FC74+982132]
	Ordinal0 [0x003A46B4+1132212]
	Ordinal0 [0x003B4812+1198098]
	Ordinal0 [0x003A42B6+1131190]
	Ordinal0 [0x0037E860+976992]
	Ordinal0 [0x0037F756+980822]
	GetHandleVerifier [0x0074CC62+2510274]
	GetHandleVerifier [0x0073F760+2455744]
	GetHandleVerifier [0x0056EABA+551962]
	GetHandleVerifier [0x0056D916+547446]
	Ordinal0 [0x00475F3B+1990459]
	Ordinal0 [0x0047A898+2009240]
	Ordinal0 [0x0047A985+2009477]
	Ordinal0 [0x00483AD1+2046673]
	BaseThreadInitThunk [0x7675FA29+25]
	RtlGetAppContainerN

In [13]:
len(driver.find_elements(by=By.CSS_SELECTOR, value='article.prd._fb.col.c-prd'))

48

In [17]:
for i in driver.find_elements(by=By.CSS_SELECTOR, value='article.prd._fb.col.c-prd'):
    i
    print(i.find_element(by=By.CSS_SELECTOR, value='h3.name').text)
    print(i.find_element(by=By.CSS_SELECTOR, value='div.prc').text)
    break

Samsung Galaxy A03 Core - 4G - 6.5" - RAM 2Go - ROM 32Go - Caméra 13+2MP - 5000mAh- Noir
48 900 FCFA


In [ ]:
articles[0]

In [ ]:
list_of_dfs = []
for category, articles in data:
    df = pd.DataFrame(articles)
    df.to_csv( os.path.join( "data", f"{category}.csv"), index=False  )
    
    df['main_category'] = category
    list_of_dfs.append(df)
    

global_df = pd.concat( list_of_dfs, axis=0)
global_df.to_csv( os.path.join( "data", f"{category}.csv"), index=False  )

In [11]:
driver.close()